In [3]:
# TODO - checar se pasta e dados existem 
# TODO2 nao ficar com o nome dataset_eeg_cafe2022 dentro de data, dados direto
!gsutil -m cp -r gs://dataset_eeg_cafe2022 ./

Copying gs://dataset_eeg_cafe2022/Y_All.xlsx...
Copying gs://dataset_eeg_cafe2022/Formato dos dados.docx...
/ [0 files][    0.0 B/  8.2 KiB]                                                
/ [0 files][    0.0 B/ 20.8 KiB]                                                
Copying gs://dataset_eeg_cafe2022/RenamedFiles/alpha_11.csv...
Copying gs://dataset_eeg_cafe2022/RenamedFiles/alpha_10.csv...
/ [0 files][    0.0 B/  7.0 MiB]                                                
/ [0 files][    0.0 B/593.8 MiB]                                                
Copying gs://dataset_eeg_cafe2022/RenamedFiles/alpha_16.csv...
Copying gs://dataset_eeg_cafe2022/RenamedFiles/alpha_18.csv...
/ [0 files][    0.0 B/593.8 MiB]                                                
/ [0 files][    0.0 B/593.8 MiB]                                                
Copying gs://dataset_eeg_cafe2022/RenamedFiles/alpha_8.csv...
Copying gs://dataset_eeg_cafe2022/RenamedFiles/alpha_1.csv...
/ [0 files][    0.0 B/593.8 Mi

In [4]:
import os, sys
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import plotly.express as px

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [14]:
csv_files_path = './dataset_eeg_cafe2022/RenamedFiles'
csv_files = [file for file in os.listdir(csv_files_path) if '.csv' in file]

if len(csv_files) == 0:
    print('Sem dados no diretório especificado.')
    exit()

In [15]:
csv_files

['alpha_1.csv',
 'alpha_10.csv',
 'alpha_11.csv',
 'alpha_12.csv',
 'alpha_14.csv',
 'alpha_16.csv',
 'alpha_17.csv',
 'alpha_18.csv',
 'alpha_19.csv',
 'alpha_21.csv',
 'alpha_3.csv',
 'alpha_4.csv',
 'alpha_5.csv',
 'alpha_6.csv',
 'alpha_7.csv',
 'alpha_8.csv',
 'alpha_9.csv',
 'cafe-1_1.csv',
 'cafe-1_10.csv',
 'cafe-1_11.csv',
 'cafe-1_12.csv',
 'cafe-1_13.csv',
 'cafe-1_14.csv',
 'cafe-1_16.csv',
 'cafe-1_17.csv',
 'cafe-1_18.csv',
 'cafe-1_19.csv',
 'cafe-1_2.csv',
 'cafe-1_21.csv',
 'cafe-1_3.csv',
 'cafe-1_4.csv',
 'cafe-1_5.csv',
 'cafe-1_6.csv',
 'cafe-1_7.csv',
 'cafe-1_8.csv',
 'cafe-1_9.csv',
 'cafe-2_1.csv',
 'cafe-2_10.csv',
 'cafe-2_11.csv',
 'cafe-2_12.csv',
 'cafe-2_13.csv',
 'cafe-2_14.csv',
 'cafe-2_16.csv',
 'cafe-2_17.csv',
 'cafe-2_18.csv',
 'cafe-2_19.csv',
 'cafe-2_2.csv',
 'cafe-2_21.csv',
 'cafe-2_3.csv',
 'cafe-2_4.csv',
 'cafe-2_5.csv',
 'cafe-2_6.csv',
 'cafe-2_7.csv',
 'cafe-2_8.csv',
 'cafe-2_9.csv',
 'chimp_1.csv',
 'chimp_10.csv',
 'chimp_11.csv',
 'c

In [17]:
df = pd.read_csv(f'{csv_files_path}/{csv_files[0]}', delimiter=',')
df.head()

,Unnamed: 0,Index,Fp1,Fp2,C3,C4,P7,P8,O1,O2,Timestamp
0,0,0.0,140700.744474,138498.292982,119123.398557,132000.439704,133360.364540,131335.162382,126233.533172,122703.723336,0.000000
1,1,1.0,-12271.979424,-17941.946440,-47641.715126,13162.696441,41421.962252,12070.701965,23113.223685,27872.602984,0.003998
2,2,2.0,-101716.016438,-106947.643393,-135590.174567,-105261.696012,-68590.820562,-97006.503881,-84496.992468,-76859.893126,0.007021
3,3,3.0,59862.553759,55528.975192,11675.836048,3385.149346,12343.013268,12430.341533,5931.683592,4503.295362,0.010017
4,4,4.0,148910.540212,146852.771563,125499.323070,130107.850445,129113.108410,130707.525397,123300.470209,119036.249105,0.014017


In [19]:
canais = df.columns.to_list()[2:10]
canais

['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

In [27]:
def plot_sinal(arquivo, canal):
    df = pd.read_csv(f'{csv_files_path}/{arquivo}', delimiter=',')
    t = df['Timestamp'].to_numpy()
    c = df[canal].to_numpy()
    fig = px.line(x=t, y=c)
    fig.show()
    print(t.max())

In [28]:
interact(plot_sinal, arquivo=csv_files, canal=canais)

interactive(children=(Dropdown(description='arquivo', options=('alpha_1.csv', 'alpha_10.csv', 'alpha_11.csv', …

<function __main__.plot_sinal(arquivo, canal)>

## TODO reunião (26/01)

* Criar um novo csv arrumado
    * Arrumar a escala de tempo de todos os dados
    * remover todos os other (canais e tempo)
    * remover parte (ou total) de sinais com ruido

In [ ]:
df2 = df[df['Timestamp'] > 50]